Requires PEAR
https://cme.h-its.org/exelixis//web/software/pear/doc.html

In [6]:
import os, glob, re
import pandas as pd
from pathlib import Path

In [42]:
RUN_TO_MERGE=2


if RUN_TO_MERGE==1:
    path_prefix = '../Data/run1/Unaligned_1234b_PF_mm1/'
    output_path = '../Data/run1/Parsed_data'
    manifest_path=f'{path_prefix}FC_04495_1234b_PF_mm1_pogdengmc_manifest.csv'

if RUN_TO_MERGE==2:
    path_prefix = '../Data/run2/Unaligned1234b_PF_mm1/'
    output_path = '../Data/run2/Parsed_data'
    manifest_path= f'{path_prefix}FC_04749_1234b_PF_mm1_pogdengmc_manifest.csv' 

Path(output_path).mkdir(parents=True, exist_ok=True)

In [55]:
fastq_path=f'{path_prefix}Data/Project_pogdengmc/'
merge_path= f'{output_path}/merged'
# use this to put 10k lines into new files
#for f in *.bz2; do bzcat $f | head -n 10000 > ../head/${f%%.*}.fastq; done

# for using the head files (faster for debug)
use_head = True
if use_head:
    merge_path= f'{output_path}/merged_head'
Path(merge_path).mkdir(parents=True, exist_ok=True)


In [56]:
# load the r1 file names
if fastq_path[-1] is not '/': fastq_path = fastq_path + '/' # add trailing / if needed
files=os.listdir(fastq_path)

In [57]:
files[:12]

['LIB037474_GEN00149021_S8_L004_R1.fastq.bz2',
 'LIB037474_GEN00149021_S8_L003_R1.fastq.bz2',
 'LIB037474_GEN00149027_S14_L001_R1.fastq.indexCount.tab',
 'LIB037474_GEN00149026_S13_L003_R2.fastq.bz2',
 'LIB037474_GEN00149020_S7_L001_R1.fastq.indexCount.tab',
 'LIB037474_GEN00149019_S6_L002_R1.fastq.indexCount.tab',
 'LIB037474_GEN00149022_S9_L004_R2.fastq.bz2',
 'LIB037474_GEN00149026_S13_L004_R2.fastq.bz2',
 'LIB037474_GEN00149022_S9_L003_R2.fastq.bz2',
 'LIB037474_GEN00149018_S5_L004_R1.fastq.indexCount.tab',
 'LIB037474_GEN00149014_S1_L003_R1.fastq.indexCount.tab',
 'LIB037474_GEN00149020_S7_L004_R2.fastq.bz2']

In [58]:
r1_files = glob.glob(fastq_path + '*R1.fastq*.bz2')
r1_files[:2]

['../Data/20190515GAS1_nextSeq/Unaligned1234b_PF_mm1/Data/Project_pogdengmc/LIB037474_GEN00149021_S8_L004_R1.fastq.bz2',
 '../Data/20190515GAS1_nextSeq/Unaligned1234b_PF_mm1/Data/Project_pogdengmc/LIB037474_GEN00149021_S8_L003_R1.fastq.bz2']

In [59]:
# get the sample info from the manifest file
info = pd.read_csv(manifest_path)
info.columns = [x.strip().replace(' ', '') for x in info.columns] # remove spaces from column names
info.columns

Index(['FC_ID', 'RunFolder', 'Lane', 'LibraryPoolID', 'LibraryPoolName',
       'LibraryId', 'LibraryName', 'Index', 'IndexName'],
      dtype='object')

In [60]:
info

,FC_ID,RunFolder,Lane,LibraryPoolID,LibraryPoolName,LibraryId,LibraryName,Index,IndexName
0,FC_04749,190312_NB501673_0439_AHYG2YBGX9,1,LIB037474,20181129_GAS,GEN00149014,S001_EK269_GAS1_p1_rep1a_plasmid,TGTACCTT-TATAGCCT,1_13-2_3
1,FC_04749,190312_NB501673_0439_AHYG2YBGX9,1,LIB037474,20181129_GAS,GEN00149015,S002_EK269_GAS1_p1_rep1b_plasmid,TCTGCTGT-TATAGCCT,1_14-2_3
2,FC_04749,190312_NB501673_0439_AHYG2YBGX9,1,LIB037474,20181129_GAS,GEN00149016,S003_EK269_GAS1_p1_rep1c_plasmid,TTGGAGGT-TATAGCCT,1_15-2_3
3,FC_04749,190312_NB501673_0439_AHYG2YBGX9,1,LIB037474,20181129_GAS,GEN00149017,S004_EK269_GAS1_p1_rep1d_plasmid,TGATACGT-TATAGCCT,1_16-2_3
4,FC_04749,190312_NB501673_0439_AHYG2YBGX9,1,LIB037474,20181129_GAS,GEN00149018,S005_EK269_GAS1_v3_rep1a_virus,TGTCTATC-ATAGAGGC,1_5-2_4
5,FC_04749,190312_NB501673_0439_AHYG2YBGX9,1,LIB037474,20181129_GAS,GEN00149019,S006_EK269_GAS1_v3_rep1b_virus,TTCGCACC-ATAGAGGC,1_6-2_4
6,FC_04749,190312_NB501673_0439_AHYG2YBGX9,1,LIB037474,20181129_GAS,GEN00149020,S007_EK269_GAS1_v3_rep1c_virus,GTAAGGTG-ATAGAGGC,1_7-2_4
7,FC_04749,190312_NB501673_0439_AHYG2YBGX9,1,LIB037474,20181129_GAS,GEN00149021,S008_EK269_GAS1_v3_rep1d_virus,GGATTAGG-ATAGAGGC,1_8-2_4
8,FC_04749,190312_NB501673_0439_AHYG2YBGX9,1,LIB037474,20181129_GAS,GEN00149022,S009_EK269_GAS1_v4_rep2a_virus,ATCACGTT-ATAGAGGC,1_9-2_4
9,FC_04749,190312_NB501673_0439_AHYG2YBGX9,1,LIB037474,20181129_GAS,GEN00149023,S010_EK269_GAS1_v4_rep2b_virus,CTTGTACT-ATAGAGGC,1_10-2_4


In [61]:
# testing the RE
p = re.compile('_(?P<id>GEN[0-9]+)_S\d*_L(?P<lane>\d+)_')
tag = p.search('LIB029294_GEN00100248_S41_L003_R1.fastq')
tag.group('id','lane')

('GEN00100248', '003')

In [62]:
def convert_names(r1_files, manifest):
    new_names = list()
    p = re.compile('_(?P<id>GEN[0-9]+)_S\d*_L(?P<lane>\d+)_')    
    for r1 in r1_files:
        tags = p.search(r1)
        sample_id = tags.group('id')
        sample_name = manifest.loc[manifest.LibraryId==sample_id, 'LibraryName'].tolist()[0]
        lane = int(tags.group('lane'))
        new_name = '{sn}-L{n}'.format(sn=sample_name, n=lane, sid=sample_id)        
        new_names.append(new_name)        
    return new_names
sample_names = convert_names(r1_files, info)
sample_names[:5]

['S008_EK269_GAS1_v3_rep1d_virus-L4',
 'S008_EK269_GAS1_v3_rep1d_virus-L3',
 'S008_EK269_GAS1_v3_rep1d_virus-L2',
 'S014_EK269_GAS1_v5_rep3b_virus-L2',
 'S007_EK269_GAS1_v3_rep1c_virus-L1']

In [ ]:
def merge_reads(r1_files, sample_names, merge_path, force_redo=False):
    PEAR_PATH = 'pear'    
    if merge_path[-1] is not '/': merge_path = merge_path + '/' # add trailing / if needed
    for r1, name in zip(r1_files, sample_names):
        r2 = r1.replace('R1.fastq.bz2', 'R2.fastq.bz2')
        merge_file = merge_path + name        
        script_finished = os.path.isfile(merge_file + '.fastq.gz' )
        valid = any([x in name for x in ('GAS1',"plasmid")])
        if valid: 
            if not script_finished or force_redo:
                print("merging " + merge_file)
                pear_cmd = '{pear} -j {threads} -f {r1} -r {r2} -o {output}'.format(pear=PEAR_PATH,                                                        r1=r1,
                                                            r2=r2,
                                                            threads=4,
                                                            output=merge_file)
                print (pear_cmd)
                os.system(pear_cmd)
                # get rid of these files
                os.system('rm ' + merge_file + '.discarded*')
                os.system('rm ' + merge_file + '.unassembled*')
                # cleanup the pear output, rename and gzip the merged reads, ouput head to new file
                os.system('mv {file}.assembled.fastq {file}.fastq'.format(file=merge_file))
                os.system('head -n 200 {file}.fastq > {path}head_{name}.fastq'.format(file=merge_file, path=merge_path, name=name))
                os.system('gzip {file}.fastq'.format(file=merge_file))
            else:
                print("skipping " + merge_file)
merge_reads(r1_files, sample_names, merge_path)     

merging ../Data/20190515GAS1_nextSeq/Parsed_data/merged_head/S008_EK269_GAS1_v3_rep1d_virus-L4
pear -j 4 -f ../Data/20190515GAS1_nextSeq/Unaligned1234b_PF_mm1/Data/Project_pogdengmc/LIB037474_GEN00149021_S8_L004_R1.fastq.bz2 -r ../Data/20190515GAS1_nextSeq/Unaligned1234b_PF_mm1/Data/Project_pogdengmc/LIB037474_GEN00149021_S8_L004_R2.fastq.bz2 -o ../Data/20190515GAS1_nextSeq/Parsed_data/merged_head/S008_EK269_GAS1_v3_rep1d_virus-L4
merging ../Data/20190515GAS1_nextSeq/Parsed_data/merged_head/S008_EK269_GAS1_v3_rep1d_virus-L3
pear -j 4 -f ../Data/20190515GAS1_nextSeq/Unaligned1234b_PF_mm1/Data/Project_pogdengmc/LIB037474_GEN00149021_S8_L003_R1.fastq.bz2 -r ../Data/20190515GAS1_nextSeq/Unaligned1234b_PF_mm1/Data/Project_pogdengmc/LIB037474_GEN00149021_S8_L003_R2.fastq.bz2 -o ../Data/20190515GAS1_nextSeq/Parsed_data/merged_head/S008_EK269_GAS1_v3_rep1d_virus-L3
merging ../Data/20190515GAS1_nextSeq/Parsed_data/merged_head/S008_EK269_GAS1_v3_rep1d_virus-L2
pear -j 4 -f ../Data/20190515GAS1_n